In [1]:
from pdfminer.pdfparser import PDFParser
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
import urllib.request
import urllib.parse

from io import StringIO
import pandas as pd
import numpy as np
import re
import os
import itertools


In [2]:
print(os.getcwd())


/Users/llc/Downloads/graphen/paper


This is a function which helps to convert the file to pdf format.

In [3]:
def convert_pdf_to_txt(path="./"):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    maxpages = 40
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    return text



In [5]:
papers = os.listdir('./doc')


list

In [7]:
####ADD another chunk to read the additional papers####

document = []
for paper in papers:
    dir = './doc/' + paper
    document.append(convert_pdf_to_txt(dir))

In [ ]:

add_paperfiles = os.listdir('./doc1/')
fil_paperfiles = []
for fil in add_paperfiles:
    if "page" in fil:
        fil_paperfiles.append(fil)
print(fil_paperfiles)
for fil in fil_paperfiles:
    for paperfiles in os.listdir(str("./doc1/")+str(fil)):
        document.append(convert_pdf_to_txt(str("./doc1/")+str(fil)+"/"+str(paperfiles)))



['page16', 'page20', 'page27', 'page18', 'page26', 'page19', 'page21', 'page17', 'page24', 'page23', 'page22', 'page25']


In [6]:
sorted(os.listdir("./"))
papers[0]

'Problems-of-variable-biomarker-evaluation-in-stratified-medicine-_2016_Lung-.pdf'

In [13]:
print(len(document))
np.savez("all_the_papers", text=document)

2286


In [18]:
document=np.load("all_the_papers.npz")["text"]
print(len(document))
for i in range(len(document)):
    document[i] = document[i].replace("\\n","")
processed = document

2286


Now the papers are already formatted. We are going to locate the gene names and extract three word after the first gene name appears and three words before the second gene name.


In [19]:
geneweb="https://www.genome.jp/dbget-bin/get_linkdb?-t+9+path:map05223"
x = urllib.request.urlopen(geneweb)
text=x.read()
text=str(text)


The items in dictionary d are the genes names

In [20]:
regex = 'K[0-9]{5}</a>'
result=re.finditer(regex,text)
indices = [m.start(0) for m in result]
d = dict()
for i in indices:
    s1 = text[i:i+6]
    s2 = text[i+25:i+text[i:].find(';')]
    d[s1] = s2


In [21]:
red={value:key for key,value in d.items()}


The keys in dict1 are the gene names in papers.
We are going to locate their positions in the paper

In [22]:
dict1 = {}
for ke,va in red.items():
    ke=ke.split(", ")
    for i in range(len(ke)):
        dict1[ke[i]] = va

#We need to add more gene names to this dictionary, because they appear slight differently in papers.
dict1['K-RAS']='K07827'
dict1['N-RAS']='K07827'
dict1['RAS']='K07827'
dict1['H-RAS']='K02833'
dict1['HER-2'] = 'K05083'
dict1['PLCG-1'] = 'K01116'
dict1['PLCG-2'] = 'K05859'
dict1['STAT'] = 'K11224'
dict1['MEK'] = 'K04368'



In the next chunk below, we have found another dictionary from https://www.genecards.org

In [23]:
data = pd.read_csv('GeneCards-SearchResults.csv')
gs = data['Gene Symbol']
des = data['Description']
des_len = len(des)
des_1 = []
for i in range (des_len):
    word = str(des[i]).split(" ")[0]
    des_1.append(word)
gene_dict = {}
for i in range (des_len):
    gene_dict[gs[i]] = des_1[i]


In [24]:
#WE have two dicts one is from gene cards.com we downloaded the csv file there. 
#We used webscraping techniques to download the genes related with non-small cell lung cancer
def merge_dicts(x, y):
    z = x.copy()  
    z.update(y)
    return z
merged_dict=merge_dicts(dict1,gene_dict)
print("Total number of gene names we are using is "+ str(len(merged_dict)))

Total number of gene names we are using is 16424


In [25]:
def locate_genes(text):
    orders=[]
    upp=text.upper()
    for gene in dict1.keys():
        ls=re.search(gene,upp)
        if ls is not None:
            orders.append(ls)
    return orders
print(locate_genes(processed[0]))

def Collection_of_pdfs(path="../doc"):
    checked = []
    for pdf_file in os.listdir(path):
        if pdf_file[-4:] == ".pdf":
            checked.append(pdf_file)
    return checked
#print(Collection_of_pdfs())

[<_sre.SRE_Match object; span=(38670, 38673), match='ALK'>, <_sre.SRE_Match object; span=(2759, 2762), match='RAS'>, <_sre.SRE_Match object; span=(9966, 9970), match='STAT'>]


In [26]:
dict2 = {}
for key, value in merged_dict.items():
    dict2[key] = value
for key, value in merged_dict.items():
    regex = r'[A-Z]{2,}'
    regex_match = re.findall(regex, key)
    if regex_match != []:
        dict2[regex_match[0]] = 1


In [27]:
def locate_sentence(paper):
    paper_list = paper.split('.')
    res = []

    for sentence in paper_list:
        gene=[]
        count = 0
        word_list = sentence.split(' ')
        word_set = set()

        for word in word_list:
            word = word.upper()
            if word in dict2 and word not in word_set:
                count += 1
                gene.append(word)
                word_set.add(word)
                
        if count >=2:
            res.append([sentence, gene[0], gene[1]])
            #first_gene.append(gene[0])
            #second_gene.append(gene[1])
            #sentence_protein.append(sentence)
    return res
    #return sentence_protein, first_gene, second_gene





##Append all the sentences contains the relationship of genes to sentence_list

In [28]:
sentence=[]
for papers in processed:
    sentence += locate_sentence(papers)
df_gene=pd.concat([pd.DataFrame(sentence[i]).T for i in range(len(sentence))], ignore_index=True)
df_gene.columns=["Sentence","Gene1","Gene2"]

In [29]:
df_gene.head(5)


,Sentence,Gene1,Gene2
0,"Nicholson e, Lucinda Billingham a\na Ca...",MRC,NHS
1,As a case study we evaluated how...,AS,WAS
2,In ﬁve studies ERCC1 use was pl...,ERCC1,USE
3,\nConclusions: We found large variatio...,LARGE,ERCC1
4,A review of published papers investigat...,WAS,LARGE


In [30]:
df_gene.shape
#There are 67594 sentences in total

(65049, 3)

In [31]:
def extract_relations(paper):
    paper_list = paper.split('.')
    res = []

    for sentence in paper_list:
        gene=[]
        count = 0
        word_list = sentence.split(' ')
        word_set = set()

        for word in word_list:
            word = word.upper()
            if word in dict2 and word not in word_set:
                count += 1
                gene.append(word)
                word_set.add(word)
                
        if count >=2:
            res.append([sentence[sentence.find(gene[0]+len(gene[0])),sentence.find(gene[1])], gene[0], gene[1]])

    return res

In [32]:
relations=[]
for papers in processed:
    relations += locate_sentence(papers)
df_relations=pd.concat([pd.DataFrame(relations[i]).T for i in range(len(relations))], ignore_index=True)
df_relations.columns=["Relations","Gene1","Gene2"]


##Then we use the fasttext to do the embeding for all the papers.
#The code below is what I used to create a Dataframe and label the sentences manually to indicate our clustering result